In [ ]:
# https://huggingface.co/docs/transformers/training
!pip install transformers

In [ ]:
import torch 
from transformers import AutoTokenizer, AutoModel, AutoConfig
import os

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# PyTorch models inherit from torch.nn.Module
class SentenceMultiClassClassifier(nn.Module):
    def __init__(self,number_class, pretrained_model):
        super(SentenceMultiClassClassifier, self).__init__()
        self.number_class = number_class
        #self.pretrained = AutoModel.from_pretrained(pretrained_model).to(device)
        self.pretrained = AutoModel.from_pretrained(pretrained_model,config=AutoConfig.from_pretrained(pretrained_model, output_attentions=True,output_hidden_states=True))

        #self.dropout = nn.Dropout(0.5) 
        #self.fc1 = nn.Linear(768, 1200)
        #self.fc2 = nn.Linear(1200, 1400)
        #self.fc3 = nn.Linear(1400, number_class)
        
        self.linear = nn.Linear(768, number_class)
        self.layeroutput = torch.nn.Sigmoid()



    def forward(self, input_ids, token_type_ids, attention_mask):            
        output_pretrained = self.pretrained(input_ids, token_type_ids, attention_mask)
        # Freeze the BERT parameters
        for param in self.pretrained.parameters():
            param.requires_grad = False
            
        #x = F.relu(self.fc1(output_pretrained.last_hidden_state[:,0,:].view(-1,768)))
        #x = self.dropout(x)
        #x = F.relu(self.fc2(x))
        #x = self.dropout(x)
        
        x = output_pretrained.last_hidden_state[:,0,:].view(-1,768)
        x = self.linear(x)
        x = self.layeroutput(x)
        return x

In [ ]:
TOKENIZER_MODEL = "bert-base-multilingual-uncased"
device = "cuda" if torch.cuda.is_available() else "cpu"
pretrained_model = "bert-base-multilingual-uncased"

In [ ]:
def model_fn(model_dir):   
    NUM_CLASS = 11
    tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_MODEL)
    model = SentenceMultiClassClassifier(NUM_CLASS,TOKENIZER_MODEL)
    with open(os.path.join(model_dir, 'model.pth'), 'rb') as f:
        #model = torch.jit.load(f)
        model.load_state_dict(torch.load(f))

    model.eval()
    return model.to(device), tokenizer

In [ ]:
def predict_fn(text, model_and_tokenizer):
    # destruct model and tokenizer
    model, tokenizer = model_and_tokenizer    
    
    # Tokenize sentences
    tokenized_text = tokenizer(text, max_length = 128, padding="max_length", return_tensors="pt")
    tokenized_text['input_ids'] = tokenized_text['input_ids'].to(device)
    tokenized_text['token_type_ids'] = tokenized_text['token_type_ids'].to(device)
    tokenized_text['attention_mask'] = tokenized_text['attention_mask'].to(device)

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**tokenized_text)

    # return dictonary, which will be json serializable
    return model_output.tolist()

In [ ]:
model_and_tokenizer = model_fn("./model")

In [ ]:
text = "I am so happy about this situation"

In [ ]:
results = predict_fn(text, model_and_tokenizer)
results

## Evaluation

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
df = pd.read_csv("./dataset/sem_eval_2018_task_1_test.csv")

In [ ]:
df.head(5)

In [ ]:
label_name = [label for label in df.columns if label not in ['ID', 'Tweet']]
id2label = {idx:label for idx, label in enumerate(label_name)}
id2label

In [ ]:
text_tweet = df.loc[:,["Tweet"]].values.tolist()

In [ ]:
text_tweet[0]

In [ ]:
results = predict_fn(text_tweet[0], model_and_tokenizer)
results

In [ ]:
def proba_to_label(prediction_proba,id2label,threshold):
    prediction_label = []
    for index in range(len(prediction_proba)):
        if prediction_proba[index] > threshold:
            prediction_label.append(id2label[index])
            
    return prediction_label

In [ ]:
print(proba_to_label(results[0],id2label,0.5))

In [ ]:
label_index = df.iloc[:,2:].astype(int)
label_index.head(5)

In [ ]:
label_index = label_index.to_numpy()
label_index

In [ ]:
predictions_proba = np.zeros_like(label_index,dtype=float)

In [ ]:
for index in range(len(text_tweet)):
    result = predict_fn(text_tweet[index], model_and_tokenizer)
    result = np.squeeze(np.array(result), axis=0)    
    predictions_proba[index,:] = result    

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
def multi_label_metrics(labels, predictions_proba, threshold = 0.5):
    y_true = labels

    predictions = np.zeros_like(labels)
    predictions[np.where(predictions_proba >= threshold)] = 1
    
    y_pred = predictions
    precision = precision_score(y_true, y_pred, average='micro')
    recall = recall_score(y_true, y_pred, average='micro')
    f1_micro_average = f1_score(y_true, y_pred, average='micro')
    
    metrics = {'precision': precision,
               'recall': recall,
               'f1': f1_micro_average}
    return metrics

In [ ]:
metrics = multi_label_metrics(label_index, predictions_proba, threshold = 0.5)

In [ ]:
metrics